In [1]:
# ch5/safe_driver_prediction_ensemble.ipynb

import numpy as np
import pandas as pd

# 데이터 경로
data_path = '/kaggle/input/porto-seguro-safe-driver-prediction/'

train = pd.read_csv(data_path + 'train.csv', index_col='id')
test = pd.read_csv(data_path + 'test.csv', index_col='id')
submission = pd.read_csv(data_path + 'sample_submission.csv', index_col='id')

In [2]:
all_data = pd.concat([train, test], ignore_index=True)
all_data = all_data.drop('target', axis=1) # 타깃 값 제거

all_features = all_data.columns # 전체 피처

### 결측값 개수를 파생 변수로 생성

In [3]:
# '데이터 하나당 결측값 개수'를 파생 변수로 추가
all_data['num_missing'] = (all_data==-1).sum(axis=1)

# 명목형 피처, 태그에 calc가 달린 피처를 제외한 피처
remaining_features = [feature for feature in all_features \
                      if ('cat' not in feature and 'calc' not in feature)] 
# num_missing을 remaining_features에 추가
remaining_features.append('num_missing')

### 명목형 피처 원-핫 인코딩 적용

In [4]:
from sklearn.preprocessing import OneHotEncoder

cat_features = [feature for feature in all_features if 'cat' in feature] # 명목형 피처

# 원-핫 인코딩 적용
onehot_encoder = OneHotEncoder()
encoded_cat_matrix = onehot_encoder.fit_transform(all_data[cat_features]) 

### `ind` 변수의 고유값을 조합한 파생 변수 `mix_ind` 생성

In [5]:
# 태그에 ind가 달린 피처
ind_features = [feature for feature in all_features if 'ind' in feature]

is_first_feature=True
for ind_feature in ind_features:
    if is_first_feature:
        all_data['mix_ind'] = all_data[ind_feature].astype(str)+'_'
        is_first_feature = False
    else:
        all_data['mix_ind'] += all_data[ind_feature].astype(str)+'_'

In [6]:
cat_count_features = []
for feature in cat_features+['mix_ind']:
    val_counts_dict = all_data[feature].value_counts().to_dict()
    all_data[f'{feature}_count'] = all_data[feature].apply(lambda x: 
                                                           val_counts_dict[x])
    cat_count_features.append(f'{feature}_count')

In [7]:
from scipy import sparse

drop_features = ['ps_ind_14', 'ps_ind_10_bin','ps_ind_11_bin', 
                 'ps_ind_12_bin','ps_ind_13_bin','ps_car_14']

# remaining_features, cat_count_features에서 drop_features를 제거한 데이터
all_data_remaining = all_data[remaining_features+cat_count_features].drop(drop_features, axis=1)

# 데이터 합치기
all_data_sprs = sparse.hstack([sparse.csr_matrix(all_data_remaining),
                               encoded_cat_matrix],
                              format='csr')

In [8]:
num_train = train.shape[0] # 훈련 데이터 개수

# 훈련 데이터와 테스트 데이터 나누기
X = all_data_sprs[:num_train]
X_test = all_data_sprs[num_train:]

y = train['target'].values

In [9]:
def eval_gini(y_true, y_pred):
    # 실제 값과 예측 값의 크기가 같은지 확인 (값이 다르면 오류 발생)
    assert y_true.shape == y_pred.shape

    n_samples = y_true.shape[0] # 데이터 개수
    L_mid = np.linspace(1 / n_samples, 1, n_samples) # 대각선 값

    # 1) 예측 값에 대한 지니계수
    pred_order = y_true[y_pred.argsort()] # y_pred 크기순으로 y_true 값 정렬
    L_pred = np.cumsum(pred_order) / np.sum(pred_order) # 로렌츠 곡선
    G_pred = np.sum(L_mid - L_pred)# 예측 값에 대한 지니계수

    # 2) 예측이 완벽할 때 지니계수
    true_order = y_true[y_true.argsort()] # y_true 크기순으로 y_true 값 정렬
    L_true = np.cumsum(true_order) / np.sum(true_order) # 로렌츠 곡선
    G_true = np.sum(L_mid - L_true) # 예측이 완벽할 때 지니계수

    # 정규화된 지니계수
    return G_pred / G_true

In [10]:
def gini_lgb(preds, dtrain):
    labels = dtrain.get_label()
    return 'gini', eval_gini(labels, preds), True

In [11]:
def gini_xgb(preds, dtrain):
    labels = dtrain.get_label()
    return 'gini', eval_gini(labels, preds)

In [12]:
from sklearn.model_selection import StratifiedKFold

# Stratified K 폴드 교차검증기 생성
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=1991)

In [13]:
max_params_lgb = {'bagging_fraction': 0.8043696643500143,
 'feature_fraction': 0.6829323879981047,
 'lambda_l1': 0.9264555612104627,
 'lambda_l2': 0.9774233689434216,
 'min_child_samples': 10,
 'min_child_weight': 125.68433948868649,
 'num_leaves': 28,
 'objective': 'binary',
 'learning_rate': 0.01,
 'bagging_freq': 1,
 'verbosity': 0,
 'random_state': 1991}

In [14]:
import lightgbm as lgbm

# OOF 방식으로 훈련된 모델로 검증 데이터 타깃 값을 예측한 확률을 담을 1차원 배열
oof_val_preds_lgb = np.zeros(X.shape[0]) 
# OOF 방식으로 훈련된 모델로 테스트 데이터 타깃 값을 예측한 확률을 담을 1차원 배열
oof_test_preds_lgb = np.zeros(X_test.shape[0]) 

# OOF 방식으로 모델 훈련, 검증, 예측
for idx, (train_idx, valid_idx) in enumerate(folds.split(X, y)):
    # 각 폴드를 구분하는 문구
    print('#'*40, f'Fold {idx+1} out of {folds.n_splits}', '#'*40)
    
    # 훈련용 데이터, 검증용 데이터 설정 ---①
    X_train, y_train = X[train_idx], y[train_idx] # 훈련용 데이터
    X_valid, y_valid = X[valid_idx], y[valid_idx] # 검증용 데이터

    # lgbm 데이터세트 생성 ---②
    dtrain = lgbm.Dataset(X_train, y_train) # lgbm 훈련 데이터세트
    dvalid = lgbm.Dataset(X_valid, y_valid) # lgbm 검증 데이터세트

    # Light GBM 모델 훈련 ---③
    lgb_model = lgbm.train(params=max_params_lgb, # 최적 하이퍼 파라미터
                           train_set=dtrain, # 훈련 데이터
                           num_boost_round=1500, # 부스팅 반복 횟수
                           valid_sets=dvalid, # 모델 성능 평가용 검증 데이터
                           feval=gini_lgb, # 검증용 평가지표
                           early_stopping_rounds=150, # 조기종료 조건
                           verbose_eval=100)
    
    # 모델 성능이 가장 좋을 때의 부스팅 반복 횟수 저장 ---④
    best_iter = lgb_model.best_iteration
    # 테스트 데이터를 활용해 예측 ---⑤
    oof_test_preds_lgb += lgb_model.predict(X_test, 
                                    num_iteration=best_iter)/folds.n_splits
    # 모델 성능 평가를 위한 oof 예측 ---⑥
    oof_val_preds_lgb[valid_idx] += lgb_model.predict(X_valid, num_iteration=best_iter)
    
    # oof 예측확률에 대한 정규화 지니계수 ---⑦
    gini_score = eval_gini(y_valid, oof_val_preds_lgb[valid_idx])
    print(f'Fold {idx+1} gini score: {gini_score}\n')

######################################## Fold 1 out of 5 ########################################
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.138567 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
Training until validation scores don't improve for 150 rounds
[100]	valid_0's binary_logloss: 0.153315	valid_0's gini: 0.270034
[200]	valid_0's binary_logloss: 0.152339	valid_0's gini: 0.280485
[300]	valid_0's binary_logloss: 0.151922	valid_0's gini: 0.286543
[400]	valid_0's binary_logloss: 0.15169	valid_0's gini: 0.291089
[500]	valid_0's binary_logloss: 0.151544	valid_0's gini: 0.294535
[600]	valid_0's binary_logloss: 0.151467	valid_0's gini: 0.296222
[700]	valid_0's binary_logloss: 0.151405	valid_0's gini: 0.297658
[800]	valid_0's binary_logloss: 0.151368	valid_0's gini: 0.298591
[900]	valid_0's binary_logloss: 0.151337	valid_0's gini: 0.299329
[1000]	valid_0's b

In [15]:
max_params_xgb = {'colsample_bytree': 0.8927325521002059,
 'gamma': 9.766883037651555,
 'max_depth': 7,
 'min_child_weight': 6.0577898395058085,
 'reg_alpha': 8.136089122187865,
 'reg_lambda': 1.385119327658532,
 'scale_pos_weight': 1.5142072116395773,
 'subsample': 0.717425859940308,
 'objective': 'binary:logistic',
 'learning_rate': 0.05,
 'random_state': 1991}

In [16]:
import xgboost as xgb

# OOF 방식으로 훈련된 모델로 검증 데이터 타깃 값을 예측한 확률을 담을 1차원 배열
oof_val_preds_xgb = np.zeros(X.shape[0]) 
# OOF 방식으로 훈련된 모델로 테스트 데이터 타깃 값을 예측한 확률을 담을 1차원 배열
oof_test_preds_xgb = np.zeros(X_test.shape[0]) 

# OOF 방식으로 모델 훈련, 검증, 예측
for idx, (train_idx, valid_idx) in enumerate(folds.split(X, y)):
    # 각 폴드를 구분하는 문구
    print('#'*40, f'Fold {idx+1} out of {folds.n_splits}', '#'*40)
    
    # 훈련용 데이터, 검증용 데이터 설정
    X_train, y_train = X[train_idx], y[train_idx]
    X_valid, y_valid = X[valid_idx], y[valid_idx]

    # xgb 데이터세트 생성 
    dtrain = xgb.DMatrix(X_train, y_train)
    dvalid = xgb.DMatrix(X_valid, y_valid)
    dtest = xgb.DMatrix(X_test)
    
    watchlist = [(dtrain, 'train'), (dvalid, 'valid')]

    xgb_model = xgb.train(params=max_params_xgb, 
                           dtrain=dtrain,
                           num_boost_round=1000,
                           evals=watchlist,
                          maximize=True,
                           feval=gini_xgb,
                           early_stopping_rounds=150,
                           verbose_eval=100)

    # 모델 성능이 가장 좋을 때의 부스팅 반복 횟수 저장
    best_iter = xgb_model.best_iteration
    # 테스트 데이터를 활용해 예측
    oof_test_preds_xgb += xgb_model.predict(dtest, 
                                    ntree_limit=best_iter)/folds.n_splits
    # 모델 성능 평가를 위한 oof 예측 ---⑥
    oof_val_preds_xgb[valid_idx] += xgb_model.predict(dvalid, ntree_limit=best_iter)
    
    # oof 예측확률에 대한 정규화 지니계수 ---⑦
    gini_score = eval_gini(y_valid, oof_val_preds_xgb[valid_idx])
    print(f'Fold {idx+1} gini score: {gini_score}\n')

######################################## Fold 1 out of 5 ########################################
[03:44:14] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	train-logloss:0.65258	train-gini:0.15851	valid-logloss:0.65282	valid-gini:0.15941
[100]	train-logloss:0.15549	train-gini:0.30907	valid-logloss:0.15637	valid-gini:0.28555
[200]	train-logloss:0.15314	train-gini:0.33829	valid-logloss:0.15485	valid-gini:0.29589
[300]	train-logloss:0.15240	train-gini:0.35689	valid-logloss:0.15475	valid-gini:0.29885
[400]	train-logloss:0.15174	train-gini:0.37316	valid-logloss:0.15474	valid-gini:0.29912
[500]	train-logloss:0.15119	train-gini:0.38594	valid-logloss:0.15473	valid-gini:0.29909
[600]	train-logloss:0.15069	train-gini:0.39837	valid-logloss:0.15473	valid-gini:0.29956
[700]	train-logloss:0.15024	train-gi

In [17]:
print('LightGBM OOF Gini Score:', eval_gini(y, oof_val_preds_lgb))

LightGBM OOF Gini Score: 0.2893420267793303


In [18]:
print('XGBoost OOF Gini Score:', eval_gini(y, oof_val_preds_xgb))

XGBoost OOF Gini Score: 0.28849001909297844


In [19]:
oof_test_preds = oof_test_preds_lgb * 0.6 + oof_test_preds_xgb * 0.4
submission['target'] = oof_test_preds
submission.to_csv('submission.csv')